In [1]:
import pandas as pd
df = pd.read_csv('C:\DeepLearning\chap09\data\class2.csv')
df

,Unnamed: 0,id,tissue,class,class2,x,y,r
0,0,mdb000,C,CIRC,N,535.0,475.0,192.0
1,1,mdb001,A,CIRA,N,433.0,268.0,58.0
2,2,mdb002,A,CIRA,I,NaN,NaN,NaN
3,3,mdb003,C,CIRC,B,NaN,NaN,NaN
4,4,mdb004,F,CIRF,I,488.0,145.0,29.0
5,5,mdb005,F,CIRF,B,544.0,178.0,26.0


In [2]:
df.isnull().sum()

Unnamed: 0    0
id            0
tissue        0
class         0
class2        0
x             2
y             2
r             2
dtype: int64

In [3]:
df.isnull().sum() / len(df)

Unnamed: 0    0.000000
id            0.000000
tissue        0.000000
class         0.000000
class2        0.000000
x             0.333333
y             0.333333
r             0.333333
dtype: float64

In [4]:
df = df.dropna(how='all')
print(df)

   Unnamed: 0      id tissue class class2      x      y      r
0           0  mdb000      C  CIRC      N  535.0  475.0  192.0
1           1  mdb001      A  CIRA      N  433.0  268.0   58.0
2           2  mdb002      A  CIRA      I    NaN    NaN    NaN
3           3  mdb003      C  CIRC      B    NaN    NaN    NaN
4           4  mdb004      F  CIRF      I  488.0  145.0   29.0
5           5  mdb005      F  CIRF      B  544.0  178.0   26.0


In [5]:
df1 = df.dropna()
print(df1)

   Unnamed: 0      id tissue class class2      x      y      r
0           0  mdb000      C  CIRC      N  535.0  475.0  192.0
1           1  mdb001      A  CIRA      N  433.0  268.0   58.0
4           4  mdb004      F  CIRF      I  488.0  145.0   29.0
5           5  mdb005      F  CIRF      B  544.0  178.0   26.0


In [6]:
df2 = df.fillna(0)
print(df2)

   Unnamed: 0      id tissue class class2      x      y      r
0           0  mdb000      C  CIRC      N  535.0  475.0  192.0
1           1  mdb001      A  CIRA      N  433.0  268.0   58.0
2           2  mdb002      A  CIRA      I    0.0    0.0    0.0
3           3  mdb003      C  CIRC      B    0.0    0.0    0.0
4           4  mdb004      F  CIRF      I  488.0  145.0   29.0
5           5  mdb005      F  CIRF      B  544.0  178.0   26.0


In [7]:
df['x'] = df['x'].fillna(df['x'].mean())
print(df)

   Unnamed: 0      id tissue class class2      x      y      r
0           0  mdb000      C  CIRC      N  535.0  475.0  192.0
1           1  mdb001      A  CIRA      N  433.0  268.0   58.0
2           2  mdb002      A  CIRA      I  500.0    NaN    NaN
3           3  mdb003      C  CIRC      B  500.0    NaN    NaN
4           4  mdb004      F  CIRF      I  488.0  145.0   29.0
5           5  mdb005      F  CIRF      B  544.0  178.0   26.0


In [9]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
df = pd.read_csv('C:\DeepLearning\chap09\data\diabetes.csv')
X = df[df.columns[:-1]]
y = df['Outcome']

X = X.values
y = torch.tensor(y.values)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [23]:
ms = MinMaxScaler()
ss = StandardScaler()

X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
y_train = ms.fit_transform(y_train)
y_test = ms.fit_transform(y_test)

In [24]:
class customdataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.len = len(self.X)
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    def __len__(self):
        return self.len

In [34]:
train_data =customdataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
test_data =customdataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))

train_loader = DataLoader(dataset = train_data, batch_size =64, shuffle = True)
test_loader = DataLoader(dataset = test_data, batch_size =64, shuffle = True)

In [35]:
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        self.layer_1 = nn.Linear(8, 64, bias=True)
        self.layer_2 = nn.Linear(64, 64, bias=True)
        self.layer_out = nn.Linear(64, 1, bias=True)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)

    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        return x

In [36]:
epochs = 500+1
print_epoch = 20
LEARNING_RATE = 1e-2

model = binaryClassification()
model.to(device)
print(model)
BCE = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

binaryClassification(
  (layer_1): Linear(in_features=8, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [37]:
def accuracy(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    return acc

In [38]:
for epoch in range(epochs):
    iteration_loss = 0.
    iteration_accuracy = 0.

    model.train()
    for i, data in enumerate(train_loader):
        X, y = data
        y_pred = model(X.float())
        loss = BCE(y_pred, y.reshape(-1,1).float())

        iteration_loss += loss
        iteration_accuracy += accuracy(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if(epoch % print_epoch == 0):
        print('Train: epoch: {0} - loss: {1:.5f}; acc: {2:.3f}'.format(epoch, iteration_loss/(i+1), iteration_accuracy/(i+1)))

    iteration_loss = 0.
    iteration_accuracy = 0.
    model.eval()
    for i, data in enumerate(test_loader):
        X, y = data
        y_pred = model(X.float())
        loss = BCE(y_pred, y.reshape(-1,1).float())
        iteration_loss += loss
        iteration_accuracy += accuracy(y_pred, y)
    if(epoch % print_epoch == 0):
        print('Test: epoch: {0} - loss: {1:.5f}; acc: {2:.3f}'.format(epoch,
              iteration_loss/(i+1), iteration_accuracy/(i+1)))

Train: epoch: 0 - loss: 0.67915; acc: 60.556
Test: epoch: 0 - loss: 0.66767; acc: 65.000
Train: epoch: 20 - loss: 0.48691; acc: 74.889
Test: epoch: 20 - loss: 0.55206; acc: 73.250
Train: epoch: 40 - loss: 0.44416; acc: 78.111
Test: epoch: 40 - loss: 0.52608; acc: 74.500
Train: epoch: 60 - loss: 0.43747; acc: 77.889
Test: epoch: 60 - loss: 0.52725; acc: 74.500
Train: epoch: 80 - loss: 0.35610; acc: 84.000
Test: epoch: 80 - loss: 0.52819; acc: 75.750
Train: epoch: 100 - loss: 0.36404; acc: 83.222
Test: epoch: 100 - loss: 0.49798; acc: 76.000
Train: epoch: 120 - loss: 0.43802; acc: 77.778
Test: epoch: 120 - loss: 0.52141; acc: 74.750
Train: epoch: 140 - loss: 0.53523; acc: 78.778
Test: epoch: 140 - loss: 0.54119; acc: 74.750
Train: epoch: 160 - loss: 0.50130; acc: 78.778
Test: epoch: 160 - loss: 0.52320; acc: 75.250
Train: epoch: 180 - loss: 0.36690; acc: 83.889
Test: epoch: 180 - loss: 0.51424; acc: 76.500
Train: epoch: 200 - loss: 0.36768; acc: 84.333
Test: epoch: 200 - loss: 0.52933; a